# The notebook is for some manual data scraping

In [114]:
# stdlib imports
from urllib.request import urlopen
from os.path import join as path_join

# tpl imports
import pandas as pd

In [115]:
# settings
URL_PATTERN = 'https://barttorvik.com/trank.php?year={year}&end={year}0312&csv=1#'
TEAM_URL_PATTERN = 'http://barttorvik.com/{year}_team_results.csv'
YEARS = (2008, 2022)
OUTPUT = path_join('data', 'raw_cbb.csv')

In [116]:
# read in web pages
columns = ['TEAM', 'ADJOE', 'ADJDE', 'BARTHAG', 'RECORD', 'W', 'G', 'EFG_O', 'EFG_D', 'FTR', 'FTRD', 'TOR', 'TORD', 
        'ORB', 'DRB', 'UNK1', '2P_O', '2P_D', '3P_O', '3P_D', 'UNK2', 'UNK3', 'UNK4', 'UNK5', 'UNK6', 'UNK7', 'ADJ_T', 
        'UNK8', 'UNK9', 'UNK10', 'YEAR', 'UNK11', 'UNK12', 'UNK13', 'WAB', 'UNK14', 'UNK15']
data_by_year = {}
for year in range(YEARS[0], YEARS[1]+1):
    resource = urlopen(URL_PATTERN.format(year=year))
    stats_df = pd.read_csv(resource, header=None, names=columns, index_col=False, usecols=lambda c: not c.startswith('UNK'))

    resource = urlopen(TEAM_URL_PATTERN.format(year=year))
    conf_df = pd.read_csv(resource, index_col=False, usecols=['team', 'conf'])
    conf_df.rename(columns={'team': 'TEAM', 'conf': 'CONF'}, inplace=True)

    merged_df = stats_df.merge(conf_df, how='left', on='TEAM', validate='1:1')

    data_by_year[year] = merged_df

# combine into one
combined_df = pd.concat(data_by_year.values(), ignore_index=True)


In [117]:
# save data to csv
combined_df.to_csv(OUTPUT)